In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

# Return Prediction

In [7]:
tickers = ['AAPL','NVDA','MSFT','LLY','DVA','COR','JPM','BAC','WFC',"AMZN", "NKE", "SBUX","GOOGL", "META", "VZ","BA", "MMM", "GE","PG", "KO", "PEP","XOM", "CVX", "SLB","NEE", "DUK", "SO","AMT", "SPG", "PLD","BHP", "RIO", "ECL"]
data = pd.read_csv('merged.csv', index_col = 0)
data

,AAPL,AMT,AMZN,BA,BAC,BHP,COR,CVX,DUK,DVA,...,PG_bl,PLD_bl,RIO_bl,SBUX_bl,SLB_bl,SO_bl,SPG_bl,VZ_bl,WFC_bl,XOM_bl
Date,,,,,,,,,,,,,,,,,,,,,
2019-02-27,0.003098,-0.004712,0.002866,0.020315,0.008194,-0.003671,-0.013465,0.001167,0.004500,-0.018452,...,84.384505,60.180218,32.819811,60.759566,37.267593,38.387174,133.592358,40.678132,41.342251,55.318263
2019-02-28,-0.009836,0.017032,-0.000768,0.010380,-0.010211,-0.025424,0.006160,-0.004164,0.004256,-0.000351,...,85.165728,60.498095,32.976219,61.012039,37.249433,38.570166,133.809597,40.708327,41.328620,55.855836
2019-03-01,0.010511,0.010048,0.019453,0.001500,0.007909,-0.007372,-0.003842,0.020488,-0.001450,0.002636,...,85.474682,60.627424,33.025418,61.299502,37.288458,38.633556,132.923216,40.676481,41.320874,56.233549
2019-03-04,0.005029,0.009948,0.014620,-0.017997,-0.009553,0.005332,-0.030128,0.000738,0.003909,-0.028221,...,85.618214,60.762880,33.106702,61.649085,37.303158,38.728930,132.505011,40.727640,41.318066,56.270579
2019-03-05,-0.001820,0.007624,-0.002205,-0.005940,0.001722,0.001894,-0.010189,0.009581,-0.003894,-0.042208,...,85.691880,60.860296,33.125684,62.243301,37.331114,38.832481,132.014307,40.845858,41.326191,56.521465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-25,-0.001697,0.017534,0.005610,-0.057208,0.012432,-0.007560,0.006716,0.025186,0.012587,0.028432,...,143.180758,123.590612,63.677516,90.413680,46.907322,67.003606,137.544371,36.054594,46.248249,94.586764
2024-01-26,-0.009013,-0.014585,0.008685,0.017783,0.001198,0.004214,-0.005301,0.003837,0.001462,-0.002420,...,143.021841,123.264666,63.790200,90.449849,46.918339,66.963430,137.472308,36.211315,46.200025,94.490834
2024-01-29,-0.003586,0.009225,0.013449,-0.001363,0.005384,0.000000,0.003491,-0.000402,-0.001043,0.010637,...,143.190843,123.302445,63.943230,90.697166,46.840670,66.868427,137.411745,36.480338,46.157634,94.400190


# Portfolio Construction

In [20]:
# all features
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

predicted_returns_rf_list = []
predicted_returns_lr_list = []
predicted_returns_tree_list = []

for stock in tickers:
    print(stock)
    
    features = ['GDP', 'CPI', 'Unemployment Rate', 'Federal Funds Rate',
       'Retail Sales', 'Mkt-RF', 'SMB', 'HML', 'RF', stock+'_ma50', stock+'_rsi', stock+'_macd', stock+'_macds',
        stock+'_bm', stock+'_bu',  stock+'_bl']
#     features = ['Mkt-RF', 'SMB', 'HML', 'RF', stock+'_rsi']
    
    X = data[features]
    y = data[tickers]
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    model1 = RandomForestRegressor(max_depth = 8)
    model2 = LinearRegression()
    model3 = DecisionTreeRegressor(max_depth = 4)
    model1.fit(x_train, y_train[stock])
    model2.fit(x_train, y_train[stock])
    model3.fit(x_train, y_train[stock])
    
    predictions_rf = model1.predict(x_test)
    predicted_returns_rf_list.append(pd.DataFrame(predictions_rf, index=x_test.index, columns=[stock]))
    
    predictions_lr = model2.predict(x_test)
    predicted_returns_lr_list.append(pd.DataFrame(predictions_lr, index=x_test.index, columns=[stock]))
    
    predictions_tree = model3.predict(x_test)
    predicted_returns_tree_list.append(pd.DataFrame(predictions_tree, index=x_test.index, columns=[stock]))
    
predicted_returns_rf = pd.concat(predicted_returns_rf_list, axis = 1)
predicted_returns_lr = pd.concat(predicted_returns_lr_list, axis = 1)
predicted_returns_tree = pd.concat(predicted_returns_tree_list, axis = 1)

AAPL
NVDA
MSFT
LLY
DVA
COR
JPM
BAC
WFC
AMZN
NKE
SBUX
GOOGL
META
VZ
BA
MMM
GE
PG
KO
PEP
XOM
CVX
SLB
NEE
DUK
SO
AMT
SPG
PLD
BHP
RIO
ECL


In [21]:
predicted_returns_rf.dropna().to_csv('predicted_returns_rf_depth8.csv')
predicted_returns_lr.dropna().to_csv('predicted_returns_lr.csv')
predicted_returns_tree.dropna().to_csv('predicted_returns_tree_depth4.csv')

In [22]:
predicted_returns_rf_list_ff = []
predicted_returns_lr_list_ff = []
predicted_returns_tree_list_ff = []

for stock in tickers:
    print(stock)
    
    features = ['Mkt-RF', 'SMB', 'HML', 'RF', stock+'_rsi']
    
    X = data[features]
    y = data[tickers]
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    model1 = RandomForestRegressor()
    model2 = LinearRegression()
    model3 = DecisionTreeRegressor(max_depth = 4)
    model1.fit(x_train, y_train[stock])
    model2.fit(x_train, y_train[stock])
    model3.fit(x_train, y_train[stock])
    
    predictions_rf = model1.predict(x_test)
    predicted_returns_rf_list_ff.append(pd.DataFrame(predictions_rf, index=x_test.index, columns=[stock]))
    
    predictions_lr = model2.predict(x_test)
    predicted_returns_lr_list_ff.append(pd.DataFrame(predictions_lr, index=x_test.index, columns=[stock]))
    
    predictions_tree = model3.predict(x_test)
    predicted_returns_tree_list_ff.append(pd.DataFrame(predictions_tree, index=x_test.index, columns=[stock]))
    
predicted_returns_rf_ff = pd.concat(predicted_returns_rf_list_ff, axis = 1)
predicted_returns_lr_ff = pd.concat(predicted_returns_lr_list_ff, axis = 1)
predicted_returns_tree_ff = pd.concat(predicted_returns_tree_list_ff, axis = 1)

AAPL
NVDA
MSFT
LLY
DVA
COR
JPM
BAC
WFC
AMZN
NKE
SBUX
GOOGL
META
VZ
BA
MMM
GE
PG
KO
PEP
XOM
CVX
SLB
NEE
DUK
SO
AMT
SPG
PLD
BHP
RIO
ECL


In [23]:
predicted_returns_rf_ff.dropna().to_csv('predicted_returns_rf_four_factor.csv')
predicted_returns_lr_ff.dropna().to_csv('predicted_returns_lr_four_factor.csv')
predicted_returns_tree_ff.dropna().to_csv('predicted_returns_tree_four_factor_depth4.csv')